Skew-T Radiosonde plot

$ source activate py36skewT

$ jupyter notebook


In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python')
import numpy as np
import urllib3
from bs4 import BeautifulSoup
import createFolder as cF
import matplotlib.pyplot as plt

from datetime import date
import calendar
import pandas as pd
import metpy.calc as mpcalc

import plot_skewT as skewT

In [ ]:
savetxt = 0
savefig = 0

stn = '01415'

year = '2016'
# Nov 2016
month = '11'
t = np.arange(11,19)
t = np.append(t,[28,29])

# Dec 2016
#month = '12'
#t = np.arange(21,28)

#year = '2017'
# Jan 2017
#month = '01'
#t = np.arange(2,14)
#t = np.append(t,[28,29])

# Feb 2017
#month = '02'
#t = np.arange(1,5)

hh = ['00', '12']

file_txt = '../../Data/Sounding/%s_txt/%s%s/' %(stn,year,month)
if savetxt == 1:
    cF.createFolder(file_txt)
if savefig == 1:
    fig_file = '../../Figures/Sounding/%s_fig/%s%s/' %(stn,year,month)
    cF.createFolder(fig_file)

In [ ]:
for day in t:
    # get calendar day
    my_date = date(int(year),int(month), int(day))
    calday = calendar.day_name[my_date.weekday()]
    calmon = calendar.month_abbr[int(month)]
    
    if day < 10:
        day = '0%s' %day
    else:
        day = str(day)
    
    for hour in hh:
        ############ Save txt file #######
        if savetxt == 1:
            #1) Wyoming URL to download Sounding from
            url = 'http://weather.uwyo.edu/cgi-bin/sounding?region=europe&TYPE=TEXT%3ALIST&YEAR='\
                    +year+'&MONTH='+month+'&FROM='+day+hour+'&TO='+day+hour+'&STNM='+stn
            
            #2) Remove the html tags
            http = urllib3.PoolManager()
            response = http.request('GET', url)
            soup = BeautifulSoup(response.data.decode('utf-8'),'lxml')
            data_text = soup.get_text()
            
            #3) Split the content by new line.
            splitted = data_text.split("\n", data_text.count("\n"))
        
            #4) Write this splitted text to a .txt document
            Sounding_filename = file_txt+'%s%s%s_%s.txt' %(year,month,day,hour)
            f = open(Sounding_filename, 'w')
            for line in splitted[4:]:
                f.write(line+'\n')
            f.close()
            
        ############ Read in txt file #######
        # Select col_names to be importet for the sounding plot
        col_names = ['PRES',   'HGHT',   'TEMP',   'DWPT',   'MIXR',   'DRCT',   'SKNT',   'THTA']
        header = np.arange(0,6)
        
        # Open file with everything beneath the header
        Sounding_filename = file_txt+'%s%s%s_%s.txt' %(year,month,day,hour)
        df = pd.read_table(Sounding_filename, delim_whitespace=True, skiprows=header,
                          usecols = [0, 1, 2, 3, 5, 6, 7, 8], names = col_names)
        
        ### The footer changes depending on how high the sound measured --> lines change from Radiosonde to Radiosonde
        # 1) find idx of first value matching the name 'Station'
        lines = df.index[df[col_names[0]].str.match('Station')]
        if len(lines) == 0:
            print('no file found: %s%s%s_%s' %(year,month,day,hour))
            continue
        else:
            idx = lines[0]
            
        footer = np.arange((idx+header.size),220)
        skiprow = np.append(header,footer)
        df = pd.read_table(Sounding_filename, delim_whitespace=True, skiprows=skiprow,
                          usecols = [0, 1, 2, 3, 5, 6, 7, 8], names = col_names)
        
        # prepare 1D arrays height (z), pressure (p), potential temperature (th),
        #                   water vapour mixing ratio (qv), winds (u and v) all of the same length.
        # We will pull the data out of the dataset into individual variables
        
        z = np.asarray(df['HGHT'])
        p = np.asarray(df['PRES'])
        th = np.asarray(df['THTA'])
        qv = np.asarray(df['MIXR'])
        T = np.asarray(df['TEMP'])
        Td = np.asarray(df['DWPT'])
        wind_speed = np.asarray(df['SKNT'])
        wind_dir = np.asarray(df['DRCT'])
        u, v = mpcalc.get_wind_components(wind_speed, wind_dir)
        p = p*100
        
        title = '%s, %s %s %s   %s$\,$UTC' %(calday,day,calmon,year,hour)
        
        skewT.plot_skewT(T, Td, z, p, u, v, title)
        if savefig == 1:
            figname = fig_file+year+month+day+'_'+hour+'Z.png' 
            # savefig
            plt.savefig(figname,orientation = 'portrait', papertype = 'a4')#, dpi=300,bbox_inches=0)
            if len(lines) != 0:
                print('saved: %s.png' %figname)
            plt.close()
        else:
            plt.show()
            plt.close()
        
        